In [ ]:
"""
    BGE-M3 + FAISS + Ollama-Exaone3.5  ― 완전 로컬 RAG 파이프라인
   · PDF를 벡터화해 FAISS 인덱스에 누적 저장
   · 검색(FAISS) → 생성(Ollama)까지 한 번에 수행
   · 프롬프트에 ‘문서 근거만 답하라, 없으면 모르겠다’ 규칙을 넣어 할루시네이션 최소화
"""

In [ ]:
# ollama
!curl -fsSL https://ollama.com/install.sh | sh
!ollama --version
!nohup ollama serve > log.txt 2>&1 &
!ollama pull exaone3.5:2.4b
!curl http://localhost:11434/api/tags

In [ ]:
# 0) 필수 패키지
# (노트북이면 앞에 !, 로컬쉘이면 pip install …)
!pip install -qU langchain-community langchain-text-splitters \
                faiss-cpu FlagEmbedding pymupdf requests

In [20]:
# 1) 환경 설정
from pathlib import Path
import numpy as np, requests
from FlagEmbedding import FlagModel
from langchain.embeddings.base import Embeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

In [ ]:
PDF_PATH   = "/content/drive/MyDrive/Colab Notebooks/pdf/2022개정교육과정자유학기운영안내서.pdf"
INDEX_DIR  = "faiss_bge_m3_index"
CHUNK_SIZE = 800      # 👈 길이를 줄여 전화번호 등이 안 잘리도록
CHUNK_OVER = 100
OLLAMA_API = "http://localhost:11434/api/generate"
MODEL_NAME = "exaone3.5:2.4b"

In [21]:
#  2) BGE-M3 임베딩 래퍼
class BGEEmbedding(Embeddings):
    """FlagEmbedding → LangChain compatible wrapper"""
    def __init__(self, model_name="BAAI/bge-m3", fp16=True):
        self.model = FlagModel(model_name, use_fp16=fp16)
    def _encode(self, texts):
        vecs = self.model.encode(texts, batch_size=32, max_length=8192)
        vecs = np.asarray(vecs, dtype="float32")
        vecs /= np.linalg.norm(vecs, axis=1, keepdims=True) + 1e-12
        return vecs
    def embed_documents(self, texts):  return self._encode(texts)
    def embed_query(self, text):       return self._encode([text])[0]

emb_wrapper = BGEEmbedding(fp16=True)      # GPU 없으면 fp16=False

In [ ]:
# 3) 인덱스 생성 / 업데이트
def build_faiss_index():
    loader = PyMuPDFLoader(PDF_PATH)
    docs   = loader.load()                              # page 단위
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVER
    )
    chunks  = splitter.split_documents(docs)
    if Path(INDEX_DIR).exists():
        db = FAISS.load_local(INDEX_DIR, emb_wrapper,
                              allow_dangerous_deserialization=True)
        db.add_documents(chunks)
    else:
        db = FAISS.from_documents(chunks, emb_wrapper)
    db.save_local(INDEX_DIR)
    print(f"✅ 인덱스 완료 / 총 벡터: {db.index.ntotal:,}")
    return db

In [ ]:
# 4) Ollama-Exaone 호출 함수
def ask_ollama(prompt: str, model: str = MODEL_NAME, max_tokens:int = 512):
    payload = {
        "model":   model,
        "prompt":  prompt,
        "stream":  False,
        "options": {"num_predict": max_tokens}   # 컨텍스트 길이 확보
    }
    res = requests.post(OLLAMA_API, json=payload, timeout=120)
    if res.ok:
        return res.json().get("response", "").strip()
    return f"❌ Ollama 오류: {res.status_code} {res.text}"

In [22]:
# 5) RAG 파이프라인
def ask_rag(query: str, top_k:int = 3):
    # (1) 검색
    db   = FAISS.load_local(INDEX_DIR, emb_wrapper,
                            allow_dangerous_deserialization=True)
    docs = db.similarity_search(query, k=top_k)
    context = "\n\n".join(d.page_content[:800] for d in docs)   # 과도한 길이 방지

    # (2) 지시어가 있는 프롬프트
    prompt = f"""아래 [문서] 내용에 근거해서만 한국어로 답하라.
문서에 정보가 없으면 '모르겠습니다'라고 답하라.

[문서]
{context}

[질문]
{query}

[답변]"""

    # (3) LLM 호출
    return ask_ollama(prompt)

In [23]:
# 6) 실행 예시
if __name__ == "__main__":
    #build_faiss_index()                                  # ★ 최초 1회
    q = "학교안전사고 예방 ·관리 체계의 고도화 과제에 대한 담당부서는 어디고 연락처가 어떻게 돼?"
    print("Q:", q)
    print("A:", ask_rag(q))

Q: 학교안전사고 예방 ·관리 체계의 고도화 과제에 대한 담당부서는 어디고 연락처가 어떻게 돼?


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


A: 학교안전사고 예방·관리 체계의 고도화 과제에 대한 담당부서는 **학교안전과**이며, 연락처는 **239-0841**입니다.
